In [44]:
import psycopg2 as pg

conn = pg.connect(database="NAaaS",user="postgres", password="1234", host="127.0.0.1", port="8008")
	
cursor = conn.cursor()

cursor.execute("select * from news")

if cursor.fetchone():
    print("ed")

In [45]:
import pandas as pd
df = pd.read_csv("NAaaS/Data_Generator/ResultsCleaned.csv")
# df["Link"] =
del df["CreationTime"]
# df.head()

for data in df.itertuples():
    cursor = conn.cursor()
    cursor.execute("Select name from province where name=%s",
                   (str(data[5]).upper(),))
    province = cursor.fetchone()
    
    if province:
        cursor.execute("Insert into NEWS(header, summary, details, focus_time, focus_location, province) VALUES( %s, %s, %s, %s, %s, %s);",
                       (data[2], data[3], data[4], data[1], province[0], province[0]))
        print(conn.status)
        break

    else:
        cursor.execute(
            "Select name, province from district where name=%s", (str(data[5]).upper(),))
        district = cursor.fetchone()
        if district:
            pass
        else:
            cursor.execute(
                "Select name, district from tehsil where name=%s", (str(data[5]).upper(),))
            tehsil = cursor.fetchone()
            if tehsil:
                cursor.execute(
                    "Select name, province from district where name=%s", (tehsil[1],))
                district = cursor.fetchone()
                if district:
                    pass
                    # print(tehsil, district)


SyntaxError: syntax error at or near ")"
LINE 22: ', '2022-03-19', 'SINDH', 'SINDH',);
                                           ^


In [10]:
# cursor.execute("Alter table NEWS add column summary TEXT")

df.head()

,focusTime,Header,Summary,Details,focusLocation
0,2022-03-19,Acquittal of Qandeel Baloch’s brother challeng...,State appeal says the acquittal of the convict...,LAHORE: The Punjab prosecution department on S...,multan
1,2022-03-19,Ready to give dissidents ‘benefit of doubt’: PM,Calls on PTI workers to hold peaceful demonstr...,RAWALPINDI: Prime Minister Imran Khan on Satur...,rawalpindi
2,2022-03-19,SC asks both sides to help with no-trust matter,Reference under Article 63-A to be taken up al...,ISLAMABAD: The Sup­reme Court on Saturday soug...,sindh
3,2022-03-18,Ties with Islamabad through diplomatic channel...,"When asked about Biden-Imran phone call, White...",\n\nJEN Psaki\n\n \nWASHINGTON: The Biden admi...,islamabad
4,2022-03-21,Opposition ‘walks back’ threat to disrupt OIC ...,Leaders had warned of sit-in in NA if no-trust...,ISLAMABAD: Hours after threatening to take the...,mirza
